In [1]:
from get_data import *
from learn import *
from tuning import *
from analysis import *
from binance.client import Client
import numpy as np
from sklearn.decomposition import PCA
import plotly.express as px
import pandas as pd
from backtesting import Backtest
pd.options.mode.chained_assignment = None  # default='warn'

C:\Users\howdy\anaconda3\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

### Get data

In [2]:


kline = get_klines(Client.KLINE_INTERVAL_4HOUR,'2019-01-01','2022-12-02')
OHLC = get_OHLC(kline)
tech_index = get_tech(OHLC,period=14)
OHLC_label = get_label(OHLC)
feature_idx = get_features(tech_index)


OHLC_hasFeature = OHLC_label.label_2[OHLC_label.index.isin(feature_idx.index)].shift(-1)
OHLC_hasFeature = OHLC_hasFeature[1:-6]

feature_idx_sel = feature_idx.shift(periods=1)[1:-6]
feature_cnt = feature_idx_sel.apply(pd.Series.value_counts)
print("feature:")
print(feature_cnt)
splitpoint=  OHLC_hasFeature.index.get_loc('2021-10-01 00:00:00')

# indicator
X_train = feature_idx_sel[:splitpoint].to_numpy()
y_train = OHLC_hasFeature[:splitpoint].to_numpy()
X_test = feature_idx_sel[splitpoint:].to_numpy()
y_test = OHLC_hasFeature[splitpoint:].to_numpy()

# return
OHLC_return = OHLC_label['return'][:-6].to_numpy()
OHLC_window = np.zeros((len(OHLC_return)-15,14))
for i in range(15,len(OHLC_return)):
    OHLC_window[i-15] = OHLC_return[i-15:i-1]    
OHLC_window = OHLC_window[22:]
OHLC_return_train = OHLC_window[:splitpoint]
OHLC_return_test = OHLC_window[splitpoint:]
method_name,method,method_tuning=tuning()

#backtesting
OHLC_selected = OHLC_label[OHLC_label.index.isin(OHLC_hasFeature.index)]
OHLC_backtesting = OHLC_selected[['Open','High','Low','Close','Volume']][splitpoint-1:]
feature_test = feature_idx_sel[splitpoint:]
OHLC_backtesting.index = pd.DatetimeIndex(OHLC_backtesting.index)

label:
0    4404
1    4178
Name: label_2, dtype: int64
feature:
      EMA   WMA   SMA   RSI  MACD   CCI   MTM  WILLR    KD   DMI
0.0  3994  4077  4096  4417  4329  4296  4034   4728  4613  4481
1.0  4545  4462  4443  4122  4210  4243  4505   3811  3926  4058


### PCA

In [ ]:
pca_df = copy.deepcopy(feature_idx_sel)
pca_df["label"] = OHLC_hasFeature
n_components=4
pca = PCA(n_components = n_components)
components = pca.fit_transform(feature_idx_sel)
labels = {
    str(i): f"PC{i+1}"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}
total_var = pca.explained_variance_ratio_.sum() * 100
fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(n_components),
    color=pca_df.label,
    title=f'Total Explained Variance: {total_var:.2f}%',
)
fig.update_traces(diagonal_visible=False)
fig.show()


### Confusion Matrix

In [ ]:
basic, score = confusion(OHLC_hasFeature,feature_idx_sel,splitpoint,display=False)
basic

In [ ]:
score

### train: return

In [ ]:
return_result = training(OHLC_return_train,OHLC_return_test,y_train,y_test,
                method,method_name)
print("return: without tuning")
return_result

In [ ]:
return_result_tuning = training(OHLC_return_train,OHLC_return_test,y_train,y_test,
                method_tuning,method_name)
print("return: with tuning")
return_result_tuning

### train: indicator

In [ ]:
idc_result = training(X_train,X_test,y_train,y_test,
                method,method_name)
print("indicator: without tuning")
idc_result

In [ ]:
idc_result_tuning = training(X_train,X_test,y_train,y_test,
                method_tuning,method_name)
print("indicator: with tuning")
idc_result_tuning

### backtesting

In [ ]:
# SMA Cross
test = Backtest(OHLC_backtesting, SmaCross, cash=100000, commission=0.001,trade_on_close=False)
result = test.run()  
#test.plot(filename=f"./backtest_SMAcross.html")
result

In [ ]:
# single indicator
backtest_single_indicator = batch_test_idc(OHLC_backtesting,feature_test,y_test,name="./backtest_tech.html")
print("backtesting: single indicator")
backtest_single_indicator

In [ ]:
#return: without tuning
backtest_return_result=batch_test_learn(OHLC_backtesting,OHLC_return_train,OHLC_return_test
                                        ,y_train,y_test,method,method_name,name="./backtest_Return.html")
print("backtesting: return (no tuning)")
backtest_return_result

In [ ]:
#return: tuning
backtest_return_result_tuning=batch_test_learn(OHLC_backtesting,OHLC_return_train,OHLC_return_test
                                        ,y_train,y_test,method_tuning,method_name,name="./backtest_Return.html")
print("backtesting: return (with tuning)")
backtest_return_result_tuning

In [3]:
#indicator: without tuning
backtest_indicator_result=batch_test_learn(OHLC_backtesting,X_train,X_test
                                           ,y_train,y_test,method,method_name,name="./backtest_Indicator.html")
print("backtesting: return (no tuning)")
backtest_indicator_result

<Data i=2 (2021-10-01 08:00:00) Open=44788.42, High=47786.7, Low=44700.0, Close=47488.08, Volume=21685.65842>
<Data i=3 (2021-10-01 12:00:00) Open=47488.07, High=47886.11, Low=46763.68, Close=47363.68, Volume=14747.75492>
<Data i=4 (2021-10-01 16:00:00) Open=47363.69, High=48190.0, Low=47232.98, Close=48030.0, Volume=8403.11668>
<Data i=5 (2021-10-01 20:00:00) Open=48029.99, High=48495.0, Low=47710.01, Close=48141.61, Volume=7272.32936>
<Data i=6 (2021-10-02 00:00:00) Open=48141.6, High=48224.01, Low=47443.98, Close=47520.84, Volume=6138.50236>
<Data i=7 (2021-10-02 04:00:00) Open=47520.84, High=47900.0, Low=47430.18, Close=47725.01, Volume=4435.01689>
<Data i=8 (2021-10-02 08:00:00) Open=47725.0, High=47899.01, Low=47478.44, Close=47683.45, Volume=3471.37761>
<Data i=9 (2021-10-02 12:00:00) Open=47683.46, High=48336.59, Low=47616.8, Close=47828.52, Volume=7082.00484>
<Data i=10 (2021-10-02 16:00:00) Open=47831.94, High=48267.0, Low=47650.0, Close=48239.99, Volume=4984.54689>
<Data i=1

<Data i=505 (2021-12-24 04:00:00) Open=51030.0, High=51109.09, Low=50736.11, Close=50994.75, Volume=3195.58213>
<Data i=506 (2021-12-24 08:00:00) Open=50994.75, High=51550.0, Low=50858.0, Close=50993.88, Volume=4900.76074>
<Data i=507 (2021-12-24 12:00:00) Open=50993.89, High=51244.22, Low=50384.43, Close=50956.68, Volume=5371.31998>
<Data i=508 (2021-12-24 16:00:00) Open=50956.68, High=51810.0, Low=50735.87, Close=51126.4, Volume=6101.15529>
<Data i=509 (2021-12-24 20:00:00) Open=51126.4, High=51236.15, Low=50500.0, Close=50820.0, Volume=4273.27627>
<Data i=510 (2021-12-25 00:00:00) Open=50819.99, High=51131.25, Low=50600.0, Close=50844.14, Volume=2536.10175>
<Data i=511 (2021-12-25 04:00:00) Open=50844.13, High=51084.13, Low=50656.43, Close=50919.17, Volume=2508.51405>
<Data i=512 (2021-12-25 08:00:00) Open=50919.18, High=51009.29, Low=50555.0, Close=50682.97, Volume=2634.74027>
<Data i=513 (2021-12-25 12:00:00) Open=50682.97, High=50934.78, Low=50631.55, Close=50659.91, Volume=3162.

<Data i=840 (2022-02-18 00:00:00) Open=40515.71, High=40950.0, Low=40300.0, Close=40691.98, Volume=5300.41799>
<Data i=841 (2022-02-18 04:00:00) Open=40691.97, High=40780.64, Low=40493.31, Close=40665.81, Volume=3464.12026>
<Data i=842 (2022-02-18 08:00:00) Open=40665.81, High=40959.88, Low=40101.01, Close=40178.94, Volume=5962.75694>
<Data i=843 (2022-02-18 12:00:00) Open=40178.95, High=40799.0, Low=39616.72, Close=39832.03, Volume=15730.488>
<Data i=844 (2022-02-18 16:00:00) Open=39832.02, High=40438.27, Low=39450.0, Close=40216.8, Volume=9738.09073>
<Data i=845 (2022-02-18 20:00:00) Open=40216.79, High=40286.09, Low=39711.31, Close=39974.44, Volume=3650.04849>
<Data i=846 (2022-02-19 00:00:00) Open=39974.45, High=40307.22, Low=39915.89, Close=40125.12, Volume=2584.5293>
<Data i=847 (2022-02-19 04:00:00) Open=40125.13, High=40444.32, Low=40060.88, Close=40263.71, Volume=2279.67412>
<Data i=848 (2022-02-19 08:00:00) Open=40263.7, High=40301.11, Low=39648.6, Close=39842.54, Volume=4264

<Data i=1303 (2022-05-06 04:00:00) Open=36461.81, High=36550.0, Low=35833.01, Close=36371.94, Volume=9524.98864>
<Data i=1304 (2022-05-06 08:00:00) Open=36371.94, High=36514.63, Low=35587.1, Close=35780.84, Volume=10211.57728>
<Data i=1305 (2022-05-06 12:00:00) Open=35780.83, High=36436.2, Low=35258.0, Close=36004.34, Volume=28261.27261>
<Data i=1306 (2022-05-06 16:00:00) Open=36004.33, High=36239.4, Low=35834.03, Close=36001.35, Volume=10893.48631>
<Data i=1307 (2022-05-06 20:00:00) Open=36001.34, High=36116.81, Low=35863.59, Close=36013.77, Volume=3142.25822>
<Data i=1308 (2022-05-07 00:00:00) Open=36013.77, High=36077.98, Low=35760.0, Close=35812.21, Volume=3253.27224>
<Data i=1309 (2022-05-07 04:00:00) Open=35812.22, High=35961.05, Low=35722.24, Close=35921.6, Volume=4312.23187>
<Data i=1310 (2022-05-07 08:00:00) Open=35921.6, High=36146.3, Low=35879.96, Close=36055.86, Volume=4389.11246>
<Data i=1311 (2022-05-07 12:00:00) Open=36055.86, High=36085.5, Low=35866.0, Close=35962.47, V

<Data i=1724 (2022-07-15 08:00:00) Open=20835.38, High=21059.32, Low=20720.16, Close=20769.74, Volume=27470.72494>
<Data i=1725 (2022-07-15 12:00:00) Open=20769.74, High=21090.0, Low=20759.39, Close=20792.19, Volume=31597.1672>
<Data i=1726 (2022-07-15 16:00:00) Open=20792.19, High=21189.0, Low=20636.7, Close=21165.85, Volume=21907.5011>
<Data i=1727 (2022-07-15 20:00:00) Open=21165.85, High=21200.0, Low=20760.0, Close=20830.04, Volume=18085.215>
<Data i=1728 (2022-07-16 00:00:00) Open=20830.04, High=20865.38, Low=20600.0, Close=20738.0, Volume=14640.8498>
<Data i=1729 (2022-07-16 04:00:00) Open=20738.0, High=20764.83, Low=20478.61, Close=20546.18, Volume=14599.27645>
<Data i=1730 (2022-07-16 08:00:00) Open=20546.18, High=20685.9, Low=20539.85, Close=20616.69, Volume=10306.66166>
<Data i=1731 (2022-07-16 12:00:00) Open=20616.69, High=20957.46, Low=20588.72, Close=20943.02, Volume=19963.84945>
<Data i=1732 (2022-07-16 16:00:00) Open=20943.02, High=21588.94, Low=20921.07, Close=21192.04,

...

<Data i=1574 (2022-06-20 08:00:00) Open=20086.62, High=21022.0, Low=20023.14, Close=20801.79, Volume=26862.62188>
<Data i=1575 (2022-06-20 12:00:00) Open=20801.79, High=21090.0, Low=20364.33, Close=20824.74, Volume=20540.9217>
<Data i=1576 (2022-06-20 16:00:00) Open=20824.74, High=20832.82, Low=19832.95, Close=20072.81, Volume=16553.12261>
<Data i=1577 (2022-06-20 20:00:00) Open=20072.8, High=20784.77, Low=20001.53, Close=20573.89, Volume=12171.48997>
<Data i=1578 (2022-06-21 00:00:00) Open=20573.9, High=20783.99, Low=20348.4, Close=20614.04, Volume=14312.3475>
<Data i=1579 (2022-06-21 04:00:00) Open=20614.04, High=21333.0, Low=20474.41, Close=21120.28, Volume=20989.91077>
<Data i=1580 (2022-06-21 08:00:00) Open=21120.28, High=21500.01, Low=20911.13, Close=20962.42, Volume=19870.84399>
<Data i=1581 (2022-06-21 12:00:00) Open=20962.42, High=21723.0, Low=20853.36, Close=21392.19, Volume=24368.42434>
<Data i=1582 (2022-06-21 16:00:00) Open=21392.18, High=21550.39, Low=20870.01, Close=2088

<Data i=1967 (2022-08-24 20:00:00) Open=21709.32, High=21761.92, Low=21361.6, Close=21368.08, Volume=20679.23139>
<Data i=1968 (2022-08-25 00:00:00) Open=21368.05, High=21637.89, Low=21310.15, Close=21468.93, Volume=25661.25463>
<Data i=1969 (2022-08-25 04:00:00) Open=21468.12, High=21762.0, Low=21461.53, Close=21743.29, Volume=27720.91104>
<Data i=1970 (2022-08-25 08:00:00) Open=21744.22, High=21819.88, Low=21611.22, Close=21720.31, Volume=31114.16042>
<Data i=1971 (2022-08-25 12:00:00) Open=21720.31, High=21779.0, Low=21483.38, Close=21565.12, Volume=40638.26855>
<Data i=1972 (2022-08-25 16:00:00) Open=21563.92, High=21646.27, Low=21422.43, Close=21605.2, Volume=27793.81951>
<Data i=1973 (2022-08-25 20:00:00) Open=21607.25, High=21685.0, Low=21514.89, Close=21559.04, Volume=16987.36886>
<Data i=1974 (2022-08-26 00:00:00) Open=21559.04, High=21614.89, Low=21431.22, Close=21552.17, Volume=21810.53604>
<Data i=1975 (2022-08-26 04:00:00) Open=21552.17, High=21564.82, Low=21326.66, Close=

<Data i=2293 (2022-10-18 04:00:00) Open=19548.68, High=19706.66, Low=19524.37, Close=19664.92, Volume=37967.51537>
<Data i=2294 (2022-10-18 08:00:00) Open=19665.95, High=19673.0, Low=19470.0, Close=19563.59, Volume=40890.45>
<Data i=2295 (2022-10-18 12:00:00) Open=19563.59, High=19680.49, Low=19280.11, Close=19375.01, Volume=67510.77491>
<Data i=2296 (2022-10-18 16:00:00) Open=19375.71, High=19442.0, Low=19091.0, Close=19216.21, Volume=52650.55356>
<Data i=2297 (2022-10-18 20:00:00) Open=19217.79, High=19433.43, Low=19195.44, Close=19327.44, Volume=32027.11342>
<Data i=2298 (2022-10-19 00:00:00) Open=19327.44, High=19360.16, Low=19213.99, Close=19281.63, Volume=24838.11541>
<Data i=2299 (2022-10-19 04:00:00) Open=19281.01, High=19308.34, Low=19146.8, Close=19218.11, Volume=30838.31529>
<Data i=2300 (2022-10-19 08:00:00) Open=19218.59, High=19281.81, Low=19140.01, Close=19200.84, Volume=29796.8416>
<Data i=2301 (2022-10-19 12:00:00) Open=19201.42, High=19301.69, Low=19065.97, Close=1926

backtesting: return (no tuning)


,model test score,Return [%],Sharpe Ratio,Max. Drawdown [%],Win Rate [%],# Trades
Logistic Regression,0.494131,-14.805060,0.000000,-25.149258,44.921316,699
Gradient Boosting Classifier,0.492567,9.395054,0.328657,-26.500948,44.320988,810
Bagging Classifier,0.503912,-5.594333,0.000000,-19.722302,44.078270,971
Random Forest Classifier,0.506260,-23.555293,0.000000,-32.294231,44.107143,1120
AdaBoost Classifier,0.494523,-15.252449,0.000000,-25.149258,44.763271,697
XGBoost Classifier,0.498044,-9.738808,0.000000,-21.122035,44.892473,1116
Support Vector Classifier,0.494131,2.272411,0.145322,-19.934267,44.566545,819
K Neighbors Classifier,0.492175,-33.226606,0.000000,-47.805798,47.102804,1070


In [ ]:
#indicator: tuning
backtest_indicator_result_tuning=batch_test_learn(OHLC_backtesting,X_train,X_test
                                        ,y_train,y_test,method_tuning,method_name,name="./backtest_Indicator.html")
print("backtesting: return (with tuning)")
backtest_indicator_result_tuning